In [117]:
import random
import math
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string
from os import listdir
from collections import Counter

# Function to find the number of documents D
def loadfile(filename):
    lines = open(filename).readlines()
    dataset = list(lines)
    for i in range(len(dataset)):
        for x in dataset[i]:
            dataset[i] 
        return dataset

    
# turn a doc into clean tokens
def cleanDocument(doc):
    words = doc.split()
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]
    # remove remaining words that are not alphabetic
    words = [word for word in words if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    # filter out short tokens
    words = [word for word in words if len(word) > 1]
    # convert to lower case
    words = [word.lower() for word in words]
    return words

#separate into classes
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i].strip("\n")
        if ( vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def trainData(trainset):
    trainset=cleanDocument(str(trainset))
    wordBag = []
    wordfreq = []
    for line in trainset:
        line = line.split(' ')
        for word in line:
                wordBag.append(word)
    for w in wordBag:
        wordfreq.append(wordBag.count(w))
    frequency=sum(wordfreq)
    return wordBag

def testData(testset):
    testset=cleanDocument(str(testset))
    wordBag = []
    wordfreq = []
    for line in testset:
        line = line.split(' ')
        for word in line:
                wordBag.append(word)
    for w in wordBag:
        wordfreq.append(wordBag.count(w))
    frequency=sum(wordfreq)
    return wordBag


#positive bag with all the data
def positiveBag(separated):
    classPositive = separated['1']
    count=0
    posWordBag = []
    posWordfreq = []
    for line in classPositive:
        line=line[:-1]
        line = line.split()
        for word in line:
            count+=1
            posWordBag.append(word.lower())
    for w in posWordBag:
        posWordfreq.append(posWordBag.count(w))
#     posBag=cleanDocument(str(posWordBag))
    return posWordBag

#negative bag with all the data
def negativeBag(separated):
    classNegative = separated['0']
    count=0
    negBag=[]
    negWordBag = []
    negWordfreq = []
    for line in classNegative:
        line=line[:-1]
        line = line.split()
        for word in line:
            count+=1
            negWordBag.append(word.lower())
    for w in negWordBag:
        negWordfreq.append(negWordBag.count(w))
#     negBag=cleanDocument(str(negWordBag))
    return negWordBag



#negative bag with clean data
def neg_dict(negWordBag):
    count=0
    negdict = {}
    for word in negWordBag:
        count+=1
        if (word not in negdict):
            negdict[word]=1
        else:
            negdict[word]+=1 
    return sum(negdict.values()), negdict



#function returning count of all positive words and positive bag with unique data
def pos_dict(posWordBag):
    count=0
    posdict = {}
    for word in posWordBag:
        count+=1
        if (word not in posdict):
            posdict[word]=1
        else:
            posdict[word]+=1 
    return sum(posdict.values()), posdict


#bag of vocabularies with cleaned data
def vocabulary(trainset,vocab):
#     trainset=cleanDocument(str(trainset))
    wordBag = []
    wordfreq = []
    for line in trainset:
        line=line[:-1]
        line = line.split()
        for word in line:
            if (word not in wordBag):
                 wordBag.append(word)
    wordBag=cleanDocument(str(wordBag))
    vocab.update(wordBag)
    for w in wordBag:
        wordfreq.append(wordBag.count(w))
    frequency=sum(wordfreq)
    return frequency,wordBag


# log priors
def classPositive(separated,trainset):
    posProbability=(len(separated['1'])/len(trainset))
    return posProbability
def classNegative(separated,trainset):  
    negProbability=(len(separated['0'])/len(trainset))
    return negProbability

# get the number of all positive words
def classPositiveDenominator(posCount,vocabulary):
    posDenominator=posCount+vocabulary
    return posDenominator
def classNegativeDenominator(negCount,vocabulary):  
    negDenominator=negCount+vocabulary
    return negDenominator


def positiveProbability(posDenominator,posWordBag,wordBag,positive):
    wordProbability=[]
    count=0
    posFrequencyBag=[]
    posProbability=1
    for w in  wordBag :
        if w in posWordBag:
            posFrequencyBag.append(posWordBag.count(w))
              
    for number in posFrequencyBag:
        wordProbability.append(math.log((number+1)/posDenominator))
#     for i in wordProbability:
#         posProbability*= i
#     posProbability=positive * posProbability
    return wordProbability


def negativeProbability(negDenominator,negWordBag,testbag,negative):
    wordProbability=[]
    negFrequencyBag=[]
    negProbability=1
    for w in testbag:
        if w in negWordBag:  
            negFrequencyBag.append(negWordBag.count(w))
         
    for number in negFrequencyBag:
        wordProbability.append(math.log(number/negDenominator))
#     for i in wordProbability:
#         negProbability*= i
#     negProbability=negative*negProbability
    return wordProbability



# remember to remove this part, they are just for testing
vocab = Counter()
trainset = loadfile('yelp_labelled.txt')
testset = loadfile('amazon_cells_labelled.txt')
separated = separateByClass(trainset)
trainbag=trainData(trainset)
testbag=testData(testset)
# print(trainbag)
# print(testbag)
negWordBag= negativeBag(separated)
posWordBag= positiveBag(separated)
negCount, negdict= neg_dict(negWordBag)
posCount, posdict= pos_dict(posWordBag)
# print(negCount, negdict)
# print(posCount, posdict)
vocabulary, wordsBag= vocabulary(trainset,vocab)
positive=classPositive(separated,trainset)
print ("positive class\n" + str(positive) + "\n")
print ('___________')
negative=classNegative(separated,trainset)
print ("negative class\n" + str(negative) + "\n")
print ('___________')
posdenom=classPositiveDenominator(posCount,vocabulary)
print ("positive denominator\n" + str(posdenom) + "\n")
print ('___________')
negdenom=classPositiveDenominator(negCount,vocabulary)
print ("negative denominator\n" + str(negdenom) + "\n")


posprobability=positiveProbability(posdenom,posWordBag,wordsBag,positive)
print("positive probability\n" + str(posprobability))
print ('___________')

negprobability=negativeProbability(negdenom,negWordBag,wordsBag,negative)
print("negative probability\n" + str(negprobability))
print ('___________')




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\study\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
positive class
0.5

___________
negative class
0.5

___________
positive denominator
10225

___________
negative denominator
10829

positive probability
[-8.133978692242893, -6.834695708112632, -5.519018914206694, -5.340770682800376, -6.288152001744562, -7.8462966197911115, -8.539443800351057, -8.133978692242893, -8.539443800351057, -7.8462966197911115, -8.539443800351057, -8.539443800351057, -8.539443800351057, -8.539443800351057, -7.8462966197911115, -6.834695708112632, -3.4960186834318105, -7.440831511682948, -7.035366403574783, -5.172147970364583, -7.440831511682948, -7.623153068476902, -8.539443800351057, -7.153149439231166, -8.539443800351057, -8.133978692242893, -7.440831511682948, -6.18806854318758, -7.035366403574783, -8.539443800351057, -6.399377636854786, -7.035366403574783, -7.8462966197911115, -6.930005887916957, -6

negative probability
[-5.679065086984155, -8.596835819068435, -6.6509256700131205, -4.7152720211249965, -8.19137071096027, -8.596835819068435, -9.28998299962838, -8.596835819068435, -3.6805112044434196, -8.596835819068435, -9.28998299962838, -9.28998299962838, -7.498223530400324, -7.498223530400324, -9.28998299962838, -7.498223530400324, -8.19137071096027, -9.28998299962838, -9.28998299962838, -7.903688638508489, -5.552313381345011, -9.28998299962838, -9.28998299962838, -5.922687169641906, -9.28998299962838, -7.210541457948543, -8.19137071096027, -7.498223530400324, -9.28998299962838, -6.517394277388598, -7.903688638508489, -7.680545087194279, -6.031886461606898, -6.245460561904957, -6.031886461606898, -6.245460561904957, -9.28998299962838, -9.28998299962838, -6.58193279852617, -7.680545087194279, -7.680545087194279, -8.596835819068435, -9.28998299962838, -8.19137071096027, -8.596835819068435, -9.28998299962838, -7.210541457948543, -7.680545087194279, -8.596835819068435, -9.28998299962

In [ ]:
def testData(testset):
    testBag = []
    testfreq = []
    for line in testset:
        line = line.rsplit(' ', 1)[0]
        line = line.split(' ')
        for word in line:
            testBag.append(word)
    for w in testBag:
        testfreq.append(testBag.count(w)) 
    frequency=str(sum(testfreq))
    return testBag

def vocabulary(trainset,vocab):
    wordBag = []
    wordfreq = []
    for line in trainset:
        line=line[:-1]
        line = line.split(' ')
        for word in line:
            if (word not in wordBag):
                 wordBag.append(word)
    wordBag=cleanDocument(str(wordBag))
    vocab.update(wordBag)
    for w in wordBag:
        wordfreq.append(wordBag.count(w))
    frequency=str(sum(wordfreq))
    return frequency,wordBag

def trainData(trainset):
    trainBag = []
    trainfreq = []
    for line in trainset:
        line = line.rsplit(' ', 1)[0]
        line = line.split(' ')
        for word in line:
            if (word not in trainBag):
                 trainBag.append(word)
    trainBag=cleanDocument(str(trainBag))
    for w in trainBag:
        trainfreq.append(trainBag.count(w)) 
    frequency=str(sum(trainfreq))
    return trainBag

# #positive bag with clean data
# def positiveBag(separated):
#     classPositive = separated['1']
#     count=0
#     posWordBag = []
#     posWordfreq = []
#     for line in classPositive:
# #         line = line.rsplit(' ', 1)[0]
#         line=line[:-1]
#         line = line.split(' ')
#         for word in line:
#             count+=1
#             posWordBag.append(word)
#     for w in posWordBag:
#         posWordfreq.append(posWordBag.count(w))
#     posBag=cleanDocument(str(posWordBag))
#     return posBag

# #negative bag with clean data
# def negativeBag(separated):
#     classNegative = separated['0']
#     count=0
#     negBag=[]
#     negWordBag = []
#     negWordfreq = []
#     for line in classNegative:
# #         line = line.rsplit(' ', 1)[0]
#         line=line[:-1]
#         line = line.split(' ')
#         for word in line:
#             count+=1
#             negWordBag.append(word)
#     for w in negWordBag:
#         negWordfreq.append(negWordBag.count(w))
#     negBag=cleanDocument(str(negWordBag))
#     return negBag

#negative bag with clean data
def negativeBag(separated):
    classNegative = separated['0']
    count=0
    negBag=[]
    negWordBag = []
    negWordfreq = []
    for line in classNegative:
#         line = line.rsplit(' ', 1)[0]
        line=line[:-1]
        line = line.split(' ')
        for word in line:
            count+=1
            negWordBag.append(word)
    for w in negWordBag:
        negWordfreq.append(negWordBag.count(w))
#     negBag=cleanDocument(str(negWordBag))
    return negWordfreq

#positive bag with clean data
def positiveBag(separated):
    classPositive = separated['1']
    count=0
    posWordBag = []
    posWordfreq = []
    for line in classPositive:
        line=line[:-1]
        line = line.split(' ')
        for word in line:
            count+=1
            posWordBag.append(word)
    for w in posWordBag:
        posWordfreq.append(posWordBag.count(w))
#     posBag=cleanDocument(str(posWordBag))
    return posWordBag

#negative bag with clean data
def cleannegativeBag(separated):
    classNegative = separated['0']
    count=0
    negWordBag = {}
    for line in classNegative:
        line=line[:-1]
        line = line.split()
        for word in line:
            count+=1
            if (word not in negWordBag):
                negWordBag[word]=1
            else:
                negWordBag[word]+=1
    return count



#function returning count of all positive words and positive bag with unique data
def cleanpositiveBag(separated):
    classPositive = separated['1']
#     testset=cleanDocument(str(classPositive))
    count=0
    posWordBag = {}
    for line in classPositive:
        line=line[:-1]
        line = line.split(' ')
        for word in line:
            count+=1
            if (word not in posWordBag):
                posWordBag[word]=1
            else:
                posWordBag[word]+=1 
    return posWordBag

In [152]:
import random
import math
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string
from os import listdir
from collections import Counter

# turn a doc into clean tokens
def cleanDocument(doc):
    words = doc.split()
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]
    words = [word for word in words if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words = [word for word in words if len(word) > 1]
    words = [word.lower() for word in words]
    return words

def loadfile(filename):
    lines = open(filename).readlines()
    dataset = list(lines)
    for i in range(len(dataset)):
        for x in dataset[i]:
            dataset[i] 
        return dataset

#separate into classes
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i].strip("\n")
        if ( vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated


# get the number of all positive words
def TrainNaiveBayes(separated,trainset):
    
    wordBag = []
    wordfreq = []
    for line in trainset:
        line=line[:-1]
        line = line.split()
        for word in line:
            if (word not in wordBag):
                 wordBag.append(word)
#     wordBag=cleanDocument(str(wordBag))
    for w in wordBag:
        wordfreq.append(wordBag.count(w))
    vocabulary=sum(wordfreq)
    
    logPrior=math.log(len(separated)/len(trainset))
  
    poswordBag = []
    for line in separated:
        line=line[:-1]
        line = line.split()
        for word in line:
            if (word not in poswordBag):
                poswordBag.append(line)
    
    
    Count={}
    count=0
    for w in wordBag:
        if w in poswordBag:
            count+=1
            Count[w]+=1
        else:
            Count[w]=1
    loglikelihood={}    
    for value in Count.values():
        loglikelihood[value]=math.log((value+1)/(sum(Count.values())+1))

    return loglikelihood,logPrior,vocabulary

def TestNaiveBayes(testdoc,posloglikelihood,poslogPrior,theclass,vocabulary):
    for c in theclass:
        sum[c]=logPrior[c]
        for i in testdoc:
            word=testdoc[i]
            if word in vocabulary:
                sum[c]=sum[c]+posloglikelihood[word,c]
    return max(sum)


dataset = loadfile('yelp_labelled.txt')
separated=separateByClass(dataset)
posloglikelihood,poslogPrior,vocabulary=TrainNaiveBayes(separated['0'],trainset)
print(posloglikelihood,poslogPrior,vocabulary)
filename=loadfile('amazon_cells_labelled.txt')
print(TestNaiveBayes(filename,posloglikelihood,poslogPrior,separated['0'],vocabulary))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\study\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{1: -7.303506694902662} -0.6931471805599453 2970


NameError: name 'logPrior' is not defined

In [328]:
import random
import math
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string
from os import listdir
from collections import Counter
from ast import literal_eval

# turn a doc into clean tokens
def cleanDocument(doc):
    words = doc.split()
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]
    words = [word for word in words if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words = [word for word in words if len(word) > 1]
    words = [word.lower() for word in words]
    return words

def loadfile(filename):
    lines = open(filename).readlines()
    dataset = list(lines)
    for i in range(len(dataset)):
        for x in dataset[i]:
            dataset[i] 
        return dataset

#separate into classes
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i].strip("\n")
        if ( vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def number_of_documents(dataset):
    numlines = []
    for line in dataset:
        numlines.append(dataset.count(line))
    return numlines

def logprior(D,C):
    D=len(D)
    Ncneg=[]
    for line in C['0']:
        Ncneg.append(C['0'].count(line))        
    Ncpos=[]
    for line in C['1']:
        Ncpos.append(C['1'].count(line))
    logn=math.log(sum(Ncneg)/D)
    logp=math.log(sum(Ncpos)/D)
    
    return (int(logn)),(int(logp))
    

def likelihood(V,C):
    vocab=len(V)
    negbigdoc=[]
    for line in C['0']:
        line=line[:-1]
        line = line.split()
        for word in line:
            negbigdoc.append(word)
            
    posbigdoc=[]
    for line in C['1']:
        line=line[:-1]
        line = line.split()
        for word in line:
            posbigdoc.append(word)
    
    negOccurence=[]
    for word in V:
        if word in negbigdoc:
            negOccurence.append(negbigdoc.count(word))
    for number in negOccurence:
        neglikelihood=math.log((number+1)/(len(negbigdoc)+vocab))
    negOccurences=sum(negOccurence)
            
    posOccurence=[]
    for word in V:
        if word in posbigdoc:
            posOccurence.append(posbigdoc.count(word))
    for number in posOccurence:
        poslikelihood=math.log((number+1)/(len(posbigdoc)+vocab))
    posOccurences=sum(posOccurence)
    return neglikelihood, poslikelihood


# get the number of all positive words
def TrainNaiveBayes(D,C):
    
    Ndoc=len(D)
    Nc=[0,1]
    
    logPrior=logprior(D,C)
    
    
    V = []
    for line in D:
        line=line[:-1]
        line = line.split()
        for word in line:
            if (word not in V):
                 V.append(word)
    V=cleanDocument(str(V))
    
    loglikelihood=likelihood(V,C)
  
    return logPrior,loglikelihood,V

def TestNaiveBayes(testdoc,loglikelihood,logPrior,C,V):
    for c in C:
        sum[c]=logPrior[c]
        for i in testdoc:
            word=testdoc[i]
            if word in V:
                sum[c]=sum[c]+loglikelihood[word,c]
    print (max(sum))

file=input("Enter your file name")
D = loadfile(file)
C=separateByClass(D)

logPrior,loglikelihood,V=TrainNaiveBayes(D,C)

print(TestNaiveBayes(file,loglikelihood,logPrior,C,V))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\study\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Enter your file nameamazon_cells_labelled.txt


TypeError: tuple indices must be integers or slices, not str